In [1]:
import pandas as pd
import re
import warnings
warnings.filterwarnings("ignore")

In [2]:
# 读入数据
nlp = pd.read_csv('data/Train_Data.csv')
Test_Data = pd.read_csv('data/Test_Data.csv')

In [3]:
Test_Data.head()

,id,title,text,entity
0,f3b61b38,????#小资钱包涉嫌诈骗[超话]#李兆民～海淀区资易贷(小资钱包)诈骗证据集锦 绳之以法李...,????#小资钱包涉嫌诈骗[超话]#李兆民～海淀区资易贷(小资钱包)诈骗证据集锦 绳之以法李...,小资钱包;资易贷
1,84b12bae,蚂蚁花呗换现金现代生活飞速地发展，纵横驰骋京东白条漂洋过海。人学始知道，不学非自然。——孟郊...,蚂蚁花呗换现金现代生活飞速地发展，纵横驰骋京东白条漂洋过海。人学始知道，不学非自然。——孟郊...,京东白条;蚂蚁花呗
2,6abf4a82,NaN,嘉石榴副总经理娄振发表示，引入银行对网贷平台的资金进行全量存管，意在规避平台挪用资金，但是要...,嘉石榴
3,8d076785,宜贷网易捷金融岂止假标，供链贷诈骗1.03亿，阴阳合同，拿出借人的钱去线下放高利贷，收天价砍...,宜贷网易捷金融岂止假标，供链贷诈骗1.03亿，阴阳合同，拿出借人的钱去线下放高利贷，收天价砍...,宜贷网(沪);易捷金融;宜贷网
4,d65a1577,NaN,赵立军说，一年来，相继侦破了阿勒泰11·17炒贵金属现货金融诈骗案件、喀什5·9诈骗单位财会...,贵金属


In [4]:
# 删除训练集空数据项
print(nlp['entity'].isnull().sum())
# print(nlp[:250])
index = nlp[nlp['entity'].isnull()].index
nlp = nlp.drop(index)
print(index)
nlp = nlp.reset_index()

18
Int64Index([ 248,  261,  262,  536, 1123, 1128, 1780, 1845, 2118, 2452, 2462,
            2887, 3074, 3225, 3242, 3729, 4129, 4383],
           dtype='int64')


In [5]:
# 删除测试机的空数据项
print(Test_Data['entity'].isnull().sum())
index = Test_Data[Test_Data['entity'].isnull()].index
Test_Data = Test_Data.drop(index)
print(index)
Test_Data = Test_Data.reset_index()

16
Int64Index([  53,  463,  746,  749, 1002, 1006, 2065, 2457, 3052, 3068, 3651,
            3855, 4014, 4233, 4286, 4571],
           dtype='int64')


In [6]:
X = nlp['text']
y = nlp['negative']
Entity = nlp['entity']
X_text = Test_Data['text']
Entity_text = Test_Data['entity']

In [7]:
# 清洗数据函数
def RemovePunctuation(content):
    punctuation = '#,??,???,;;,!!,。。, '
    content = re.sub(r'[{}]+'.format(punctuation),'',str(content))
    return content.strip('?').lower()


In [8]:
# 分句子和分实体
pattern_sen = r';|\?|!|。|；|！|…'
pattern_semi = r';|；'
def CutContent(content,pattern):
    content = re.split(pattern, str(content))
    return content



In [9]:
# 将训练集句子清洗和切割
X =  X.map(lambda x :RemovePunctuation(x))
# X =  X.map(lambda x :CutContent(x,pattern_sen))


In [10]:
X[1]

'小资钱包涉嫌诈骗[超话]小资钱包[超话]迫切急待回归活命钱头号直通中纪委[超话]中纪委监察部[超话]聚焦海淀公安小资钱包爆料[超话]国务院常务会[超话]恒丰银行与小资钱包1[超话]中纪委监察部致敬国民好政策一带一路，小资钱包全体金融难民求脱贫，求致富京城一霸潘博艺，...全文：http://m.weibo.cn/3985033206/4359182960517350'

In [11]:
# 将测试集句子清洗和切割
X_text =  X_text.map(lambda x :RemovePunctuation(x))
X_text =  X_text.map(lambda x :CutContent(x,pattern_sen))

In [12]:
# 将训练集实体切割
Entity = Entity.map(lambda x :CutContent(x,pattern_semi))

In [13]:
# 将测试集实体切割
Entity_text = Entity_text.map(lambda x :CutContent(x,pattern_semi))

In [14]:
# 将训练集缺失的key_entity补充为无，避免空字符串出问题
print(Entity[0][1])
print(X[0])
nlp['key_entity'] = nlp['key_entity'].fillna('无')
# 4981

富金利
发表了博文《【富金利】9月富金利教你如何安全提高最大化收益！》网络理财时代参与互联网投资理财选择正规的理财平台是理财关键，很多人都因为没有经受住高收益的诱惑不进行理性投资而遭遇了欺骗，微弱，而对网上投资理【富金利】9月富金利教你如何安全提高最大化收益！


In [15]:
nlp['text'][0]

'????发表了博文《【富金利】9月富金利教你如何安全提高最大化收益！》网络理财时代参与互联网投资理财选择正规的理财平台是理财关键，很多人都因为没有经受住高收益的诱惑不进行理性投资而遭遇了欺骗，微弱，而对网上投资理【富金利】9月富金利教你如何安全提高最大化收益！ ?????'

In [16]:
# 将训练集数据按照实体排序
middle = nlp[:1]
middle = middle[['id','text','entity','negative','key_entity']]
k=0
for i in range(len(nlp)):
    for j in range(len(Entity[i])):
        middle.loc[k,'id']=nlp['id'][i]
        middle.loc[k,'text']=nlp['text'][i]
        middle.loc[k,'entity']=Entity[i][j]
        middle.loc[k,'negative']=nlp['negative'][i]
        middle.loc[k,'key_entity']=nlp['key_entity'][i]
        k=k+1





In [17]:
# 将测试集数据按照实体排序
middle_text = Test_Data[:1]
middle_text = middle_text[['id','text','entity']]
k=0
for i in range(len(Test_Data)):
    for j in range(len(Entity_text[i])):
        middle_text.loc[k,'id']=Test_Data['id'][i]
        middle_text.loc[k,'text']=Test_Data['text'][i]
        middle_text.loc[k,'entity']=Entity_text[i][j]
        k=k+1

In [18]:
# entity<==>key_entity
def IsNegative(keyname,name):
    if keyname.find(name) != -1:
        return '1'
    else :
        return '0'


In [19]:
# entity<==>text
def isContent(content,entity):
    if content.find(entity) != -1:
        return str(content)
    else :
        return '无'

In [20]:
# 判断训练集entity<==>key_entity
for i in range(len(middle)):
    middle.loc[i,'negative_self'] = IsNegative(middle['key_entity'][i],middle['entity'][i])

In [21]:
# print(middle['entity'])

In [22]:
# 将训练集句子清洗和切分，实体清洗
middle['text'] =  middle['text'].map(lambda x :RemovePunctuation(x))
middle['text'] =  middle['text'].map(lambda x :CutContent(x,pattern_sen))
middle['entity'] =  middle['entity'].map(lambda x :RemovePunctuation(x))

In [23]:
middle

,id,text,entity,negative,key_entity,negative_self
0,83dcefb7,"[发表了博文《【富金利】9月富金利教你如何安全提高最大化收益, 》网络理财时代参与互联网投资...",理财时代,0.0,无,0
1,83dcefb7,"[发表了博文《【富金利】9月富金利教你如何安全提高最大化收益, 》网络理财时代参与互联网投资...",富金利,0.0,无,0
2,1ad5be0d,[小资钱包涉嫌诈骗[超话]小资钱包[超话]迫切急待回归活命钱头号直通中纪委[超话]中纪委监察...,小资钱包,1.0,小资钱包;恒丰银行,1
3,1ad5be0d,[小资钱包涉嫌诈骗[超话]小资钱包[超话]迫切急待回归活命钱头号直通中纪委[超话]中纪委监察...,恒丰银行,1.0,小资钱包;恒丰银行,1
4,6dd28e9b,"[扫黑除恶诈骗犯齐聚海淀，保护伞根深蒂固，黑社会嚣张跋扈，出借人走投无路, 尊敬的中央扫黑除...",国有投资,1.0,资易贷（北京）金融信息服务有限公司;小资钱包,0
5,6dd28e9b,"[扫黑除恶诈骗犯齐聚海淀，保护伞根深蒂固，黑社会嚣张跋扈，出借人走投无路, 尊敬的中央扫黑除...",资易贷（北京）金融信息服务有限公司,1.0,资易贷（北京）金融信息服务有限公司;小资钱包,1
6,6dd28e9b,"[扫黑除恶诈骗犯齐聚海淀，保护伞根深蒂固，黑社会嚣张跋扈，出借人走投无路, 尊敬的中央扫黑除...",小资钱包,1.0,资易贷（北京）金融信息服务有限公司;小资钱包,1
7,6dd28e9b,"[扫黑除恶诈骗犯齐聚海淀，保护伞根深蒂固，黑社会嚣张跋扈，出借人走投无路, 尊敬的中央扫黑除...",资易贷,1.0,资易贷（北京）金融信息服务有限公司;小资钱包,1
8,1db87a14,[平安银行车主贷1期限长至48期2滴滴司机可做3白户逾期可做4不打家人电话5不装定位gps6...,平安银行,0.0,无,0
9,1db87a14,[平安银行车主贷1期限长至48期2滴滴司机可做3白户逾期可做4不打家人电话5不装定位gps6...,车主贷,0.0,无,0


In [24]:
# 将测试集句子清洗和切分，实体清洗
middle_text['text'] =  middle_text['text'].map(lambda x :RemovePunctuation(x))
middle_text['text'] =  middle_text['text'].map(lambda x :CutContent(x,pattern_sen))
middle_text['entity'] =  middle_text['entity'].map(lambda x :RemovePunctuation(x))

In [25]:
# middle_text

In [26]:
# 将训练集entity <==> text
for i in range(len(middle)):
    x=''
    for j in range(len(middle['text'][i])):
        x=x+isContent(middle['text'][i][j],middle['entity'][i])
    middle.loc[i,'text_self']=x

    

In [27]:
middle[['entity','negative_self','text_self']]

,entity,negative_self,text_self
0,理财时代,0,无》网络理财时代参与互联网投资理财选择正规的理财平台是理财关键，很多人都因为没有经受住高收益...
1,富金利,0,发表了博文《【富金利】9月富金利教你如何安全提高最大化收益》网络理财时代参与互联网投资理财选...
2,小资钱包,1,小资钱包涉嫌诈骗[超话]小资钱包[超话]迫切急待回归活命钱头号直通中纪委[超话]中纪委监察部...
3,恒丰银行,1,小资钱包涉嫌诈骗[超话]小资钱包[超话]迫切急待回归活命钱头号直通中纪委[超话]中纪委监察部...
4,国有投资,0,无无
5,资易贷（北京）金融信息服务有限公司,1,无尊敬的中央扫黑除恶第11督导组：我们是资易贷（北京）金融信息服务有限公司旗下小资钱包平台的...
6,小资钱包,1,无尊敬的中央扫黑除恶第11督导组：我们是资易贷（北京）金融信息服务有限公司旗下小资钱包平台的...
7,资易贷,1,无尊敬的中央扫黑除恶第11督导组：我们是资易贷（北京）金融信息服务有限公司旗下小资钱包平台的...
8,平安银行,0,平安银行车主贷1期限长至48期2滴滴司机可做3白户逾期可做4不打家人电话5不装定位gps6额...
9,车主贷,0,平安银行车主贷1期限长至48期2滴滴司机可做3白户逾期可做4不打家人电话5不装定位gps6额...


In [28]:
# 将测试集entity <==> text
for i in range(len(middle_text)):
    x=''
    for j in range(len(middle_text['text'][i])):
        x=x+isContent(middle_text['text'][i][j],middle_text['entity'][i])
    middle_text.loc[i,'text_self']=x
print(middle_text['text_self'])

0        小资钱包涉嫌诈骗[超话]李兆民～海淀区资易贷(小资钱包)诈骗证据集锦绳之以法李兆民，罪责难逃...
1        小资钱包涉嫌诈骗[超话]李兆民～海淀区资易贷(小资钱包)诈骗证据集锦绳之以法李兆民，罪责难逃...
2        蚂蚁花呗换现金现代生活飞速地发展，纵横驰骋京东白条漂洋过海人学始知道，不学非自然——孟郊《劝...
3        蚂蚁花呗换现金现代生活飞速地发展，纵横驰骋京东白条漂洋过海人学始知道，不学非自然——孟郊《劝...
4        嘉石榴副总经理娄振发表示，引入银行对网贷平台的资金进行全量存管，意在规避平台挪用资金，但是要...
5                                                        无
6        宜贷网易捷金融岂止假标，供链贷诈骗1.03亿，阴阳合同，拿出借人的钱去线下放高利贷，收天价砍...
7        宜贷网易捷金融岂止假标，供链贷诈骗1.03亿，阴阳合同，拿出借人的钱去线下放高利贷，收天价砍...
8        赵立军说，一年来，相继侦破了阿勒泰11·17炒贵金属现货金融诈骗案件、喀什5·9诈骗单位财会...
9        《中国证券报》刚刚发表了文章，对头顶fcoin币改第一币光环的qos代币进行了质疑，称其涉嫌...
10       蚂蚁花呗换现金现代生活飞速地发展，纵横驰骋京东白条漂洋过海一寸相思，一寸灰——李商隐《无题》...
11       蚂蚁花呗换现金现代生活飞速地发展，纵横驰骋京东白条漂洋过海一寸相思，一寸灰——李商隐《无题》...
12                          钱宝网张小雷因涉嫌集资诈骗罪被提起公诉未兑付本金达300亿元
13                          钱宝网张小雷因涉嫌集资诈骗罪被提起公诉未兑付本金达300亿元
14                      2、关于深圳市钱爸爸电子商务有限公司涉嫌非法吸收公众存款案的案情通报
15                      2、关于深圳市钱爸爸电子商务有限公司涉嫌非法吸收公众存款案的案情通报
16       中央巡视[超话]头号直通中纪委[超话]扫黑除恶[超话](北京)资易贷金融信息服务有限公司简称.

In [29]:
# print(middle[['text_self','entity']])
# print(middle_text[['text_self','entity']])

In [30]:
# 统计词频
from sklearn.feature_extraction.text import CountVectorizer

In [31]:
middle.to_csv('middle.csv',encoding="utf_8_sig",index=None)
middle_text.to_csv('middle_text.csv',encoding="utf_8_sig",index=None)

In [32]:
middle = pd.read_csv('middle.csv')
middle_text=pd.read_csv('middle_text.csv')

In [33]:
# 拼在一起，再进行分词和词向量划分
# 以下是提交
text = middle['text_self']
text = text.append(middle_text['text_self'])
print(len(middle['text_self']))
print(len(middle_text['text_self']))
print(len(text))

11448
11580
23028


In [34]:
# 分词和获得词向量
cv = CountVectorizer()
text = cv.fit_transform(text)
negative = middle['negative_self']
print(text)

  (0, 7428)	1
  (0, 34012)	1
  (0, 59472)	1
  (0, 38978)	1
  (0, 38767)	1
  (0, 58785)	1
  (1, 7427)	1
  (1, 26550)	1
  (1, 7428)	1
  (1, 34012)	2
  (1, 59472)	1
  (1, 38978)	1
  (1, 38767)	1
  (1, 58785)	1
  (2, 3313)	1
  (2, 2928)	1
  (2, 7787)	1
  (2, 9621)	1
  (2, 8222)	1
  (2, 21140)	1
  (2, 50540)	1
  (2, 34956)	1
  (2, 13460)	1
  (2, 39463)	1
  (2, 29523)	1
  :	:
  (23026, 37149)	1
  (23026, 58308)	1
  (23026, 61164)	1
  (23026, 1083)	1
  (23026, 62727)	1
  (23026, 61549)	1
  (23026, 7787)	2
  (23026, 9621)	1
  (23026, 8222)	1
  (23026, 21140)	1
  (23027, 3808)	1
  (23027, 9822)	1
  (23027, 30823)	1
  (23027, 57941)	1
  (23027, 7492)	1
  (23027, 37149)	1
  (23027, 58308)	1
  (23027, 61164)	1
  (23027, 1083)	1
  (23027, 62727)	1
  (23027, 61549)	1
  (23027, 7787)	2
  (23027, 9621)	1
  (23027, 8222)	1
  (23027, 21140)	1


In [35]:
# 将训练集和测试机分开
train_text = text[0:11448]
test_text = text[11448:]

In [36]:
train_text

<11448x71517 sparse matrix of type '<class 'numpy.int64'>'
	with 172984 stored elements in Compressed Sparse Row format>

In [39]:
# 基于贝叶斯定理的朴素贝叶斯分类器
# from sklearn.naive_bayes import GaussianNB,MultinomialNB,BernoulliNB
# mGB = GaussianNB()
# mGB.fit(train_text,negative)
# mGB.score(X_test,y_test)
# from sklearn.svm import SVC
# svm = SVC()
# svm.fit(train_text,negative)
# from sklearn.ensemble import RandomForestClassifier
# rf = RandomForestClassifier()
# rf.fit(train_text,negative)
# from xgboost import XGBClassifier
# xgbc = XGBClassifier()
# xgbc.fit(train_text,negative)
# from sklearn.linear_model import LogisticRegression
# molde = LogisticRegression(penalty='40')  
# molde.fit(train_text,negative)
from sklearn.neighbors import KNeighborsClassifier
clf = KNeighborsClassifier(n_neighbors=40)
clf.fit(train_text,negative)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=40, p=2,
           weights='uniform')

In [40]:
result = clf.predict(test_text)

In [41]:
print(result)

[0 0 0 ... 1 0 0]


In [42]:
middle_text['negative'] = result


In [43]:
submit = pd.read_csv('data/Submit_Example.csv')
submit.columns
middle_text.columns

Index(['id', 'text', 'entity', 'text_self', 'negative'], dtype='object')

In [44]:
print(middle_text[['id','negative']])

             id  negative
0      f3b61b38         0
1      f3b61b38         0
2      84b12bae         0
3      84b12bae         0
4      6abf4a82         0
5      8d076785         0
6      8d076785         1
7      8d076785         1
8      d65a1577         0
9      4f5344cd         0
10     a630e1f8         0
11     a630e1f8         0
12     483e80d4         0
13     483e80d4         0
14     af86add3         0
15     af86add3         0
16     8a707622         1
17     8a707622         1
18     8a707622         1
19     13792798         0
20     13792798         0
21     8d1db23b         0
22     fa1a82ad         0
23     6313d317         0
24     6313d317         1
25     6313d317         1
26     6313d317         1
27     936b4763         0
28     936b4763         0
29     e46c77f5         1
...         ...       ...
11550  b1f6fe66         0
11551  b1f6fe66         0
11552  b1f6fe66         0
11553  b1f6fe66         0
11554  b1f6fe66         0
11555  b1f6fe66         0
11556  c6f1c

In [45]:
k=0
for i in range(len(submit['id'])):
    if(k>=11580):
        break
    x=''
    while(str(middle_text['id'][k])==str(submit['id'][i])):
        
        if(str(middle_text['negative'][k])=='1'):
            submit.loc[i,'negative']=1;
            x=x+str(middle_text['entity'][k])+';'
        k=k+1
        if(k>=11580):
            break
    submit.loc[i,'key_entity']=x
        


In [46]:
print(submit)

            id  negative                    key_entity
0     f3b61b38         0                              
1     84b12bae         0                              
2     6abf4a82         0                              
3     8d076785         1                     易捷金融;宜贷网;
4     d65a1577         0                              
5     4f5344cd         0                              
6     a630e1f8         0                              
7     483e80d4         0                              
8     af86add3         0                              
9     8a707622         1  小资钱包;易贷金融;(北京)资易贷金融信息服务有限公司;
10    13792798         0                              
11    8d1db23b         0                              
12    fa1a82ad         0                              
13    6313d317         1   资易贷（北京）金融信息服务有限公司;小资钱包;资易贷;
14    936b4763         0                              
15    e46c77f5         1   资易贷（北京）金融信息服务有限公司;小资钱包;资易贷;
16    7d65264f         0                              
17    0a62

In [47]:
submit.to_csv('submit_knn简.csv',encoding="utf_8_sig",index=None)

In [44]:
# 以下为测试环节的内容
# 
# 
# 
#分词
# 以下是将全部数据输入到模型中
cv = CountVectorizer()
print(middle['text_self'])
text = cv.fit_transform(middle['text_self'])
negative = middle['negative_self']
print(text)

0        无》网络理财时代参与互联网投资理财选择正规的理财平台是理财关键，很多人都因为没有经受住高收益...
1        发表了博文《【富金利】9月富金利教你如何安全提高最大化收益》网络理财时代参与互联网投资理财选...
2        小资钱包涉嫌诈骗[超话]小资钱包[超话]迫切急待回归活命钱头号直通中纪委[超话]中纪委监察部...
3        小资钱包涉嫌诈骗[超话]小资钱包[超话]迫切急待回归活命钱头号直通中纪委[超话]中纪委监察部...
4                                                       无无
5        无尊敬的中央扫黑除恶第11督导组：我们是资易贷（北京）金融信息服务有限公司旗下小资钱包平台的...
6        无尊敬的中央扫黑除恶第11督导组：我们是资易贷（北京）金融信息服务有限公司旗下小资钱包平台的...
7        无尊敬的中央扫黑除恶第11督导组：我们是资易贷（北京）金融信息服务有限公司旗下小资钱包平台的...
8        平安银行车主贷1期限长至48期2滴滴司机可做3白户逾期可做4不打家人电话5不装定位gps6额...
9        平安银行车主贷1期限长至48期2滴滴司机可做3白户逾期可做4不打家人电话5不装定位gps6额...
10       平安银行车主贷1期限长至48期2滴滴司机可做3白户逾期可做4不打家人电话5不装定位gps6额...
11                                                 旺旺贷跑路无无
12                                                 旺旺贷跑路无无
13       为破解企业发展中的资金周转瓶颈，定南大力推行五个信贷通，截止2018年5月末，全县五个信贷通...
14                      无另一方面，发挥小额贷款公司作用，为各类企业提供贷款1.7985亿元
15                      无另一方面，发挥小额贷款公司作用，为各类企业提供贷款1.7985亿元
16       为破解企业发展中的资金周转瓶颈，定南大力推行五个信贷通，截止2018年5月末，全县五个信贷通.

In [45]:
X.iloc[-1]

'日前，依靠自建厂房按揭贷，乐清环保产业园区内的企业及其他小微企业共37户，获得了高达2.87亿元的信贷支持这是温州分行推出独创性产品自建厂房按揭贷后的首批用户依靠这个贷款产品，很多发展中的小微企业的融资问题，得到了一定程度的缓解据介绍，该产品可为小微企业解决其自主建设自用厂房、购置生产设备以及投产初期日常经营所需流动资金需求，并根据厂房建设施工进度发放贷款，为企业节约资金成本，缓解客户的融资难、融资贵问题此外，针对温州当地实际，温州分行将厂房最高抵押率提高到70%，贷款最长期限增加到10年在温州银保监分局的指导下，近两年，温州分行坚持多线联动、产品创新，累计为4个小微创业园提供项目开发贷款9.6亿元，为12个小微创业园的258家入园企业提供厂房按揭贷款支持10.17亿元华夏银行温州分行有关负责人表示，该行将扮演好中小企业金融服务商的角色，在温州银保监分局的指导下，以小微创业园为重点，加大对小微企业的金融支持，助力温州两区建设'

In [48]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(text,negative,test_size = 0.5)
# 基于贝叶斯定理的朴素贝叶斯分类器
# from sklearn.naive_bayes import GaussianNB,MultinomialNB,BernoulliNB
# mNB = MultinomialNB()
# mNB.fit(X_train,y_train)
# mNB.score(X_test,y_test)
# 梯度增强算法
# from xgboost import XGBClassifier
# xgbc = XGBClassifier()
# xgbc.fit(X_train, y_train)
# xgbc.score(X_test,y_test)

# from sklearn.ensemble import RandomForestClassifier
# rf = RandomForestClassifier()
# rf.fit(X_train, y_train)
# rf.score(X_test,y_test)
# from sklearn.linear_model import LogisticRegression
# molde = LogisticRegression()  
# molde.fit(X_train, y_train)
# result = molde.predict(X_test)
# f1_score(y_test,result,average='macro')

ValueError: Logistic Regression supports only penalties in ['l1', 'l2'], got 11.

In [49]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
X_train,X_test,y_train,y_test = train_test_split(text,negative,test_size = 0.5)
clf = KNeighborsClassifier(n_neighbors=10)
clf.fit(X_train, y_train)
result = clf.predict(X_test)
f1_score(y_test,result,average='macro')

0.6745784527880169

In [69]:
type(X_train)
import numpy as np
import scipy.sparse
my_matrix = scipy.sparse.csr_matrix(X_train)
my_array = my_matrix.A
type(list(my_array))
my_array = list(my_array)

In [55]:
# 简单神经网络
from keras import models
from keras import layers
from keras import *
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(text,negative,test_size = 0.2)
embed_dim = 128
lstm_out = 200
batch_size = 32
model = models.Sequential()
# model.add(LSTM(lstm_out, dropout_U = 0.2, dropout_W = 0.2))
# model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
model.fit(my_array, y_train)


AssertionError: 

In [155]:
x=0
for i in range(100):
    X_train,X_test,y_train,y_test = train_test_split(text,negative,test_size = 0.2)
    rf = RandomForestClassifier()
    rf.fit(X_train, y_train)
    x=x+rf.score(X_test,y_test)
print(x/100)

0.8284716157205234


In [157]:
from sklearn.svm import SVC
x=0
for i in range(100):
    X_train,X_test,y_train,y_test = train_test_split(text,negative,test_size = 0.2)
    svm = SVC(kernel='linear')
    svm.fit(X_train, y_train)
    x=x+svm.score(X_test,y_test)
print(x/10)


8.35890829694323


In [25]:
# 伯努利
GNB = BernoulliNB()
GNB.fit(X_train,y_train)
GNB.score(X_test,y_test)

0.7812227074235808

In [26]:
# negative为1
middle1 = middle[middle['negative_self']=='1']

In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer()
X_ = tf.fit_transform(middle1['text_self'])
X_1 = tf.transform(middle['text_self'])

In [28]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X_,middle1['negative_self'],test_size = 0.2)


In [29]:
X_train1,X_test1,y_train1,y_test1 = train_test_split(X_1,middle['negative_self'],test_size = 0.2)

In [30]:
tf = TfidfVectorizer()
X_train = tf.fit_transform(X_train)
X_test1 = tf.transform(X_test1)

AttributeError: lower not found

In [ ]:
from sklearn.naive_bayes import GaussianNB,MultinomialNB,BernoulliNB
mNB = MultinomialNB()
mNB.fit(X_train1,y_train1)
# mNB.score(X_test,y_test)

In [ ]:
mNB.score(X_test1,y_test1)